In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import pickle as pkl
import json


In [ ]:
source = pd.read_csv('./user_concept.csv',header=0,index_col=0)
data = source.values
print(data.shape)

In [2]:
user_list = source.index.tolist()
concept_list = source.columns.tolist()

In [3]:
user_action = []
with open('./concept_click.json','r',encoding='utf8') as action_source:
    actions = json.load(action_source)
print(len(actions))
print(actions[0])

37128
{'course_id': 'course-v1:TsinghuaX+30240184+sp', 'message': '操作系统', 'video_id': 'b2efbeccf7fb4c4cbe913e728144a644', 'user_id': '87', 'time': {'$date': 1513015162778}}


#### 根据点击历史构建user行为至少4个的用户的行为
字典类型，同时将用户列表、概念列表转化为数字表示，去信息化

In [4]:
user_action = {}
for u in range(len(user_list)):
    user = []
    for a in actions:
        if a['user_id'] == str(user_list[u]):
            if a['message'] in concept_list:
                user.append(concept_list.index(a['message']))
    user_action[u] = user
    if len(user) < 5:
        print(user)
print(len(user_action))
print(user_action[87])

[322, 33, 389, 95]
[157, 389, 481, 160]
[357, 128, 701, 93]
[175, 9, 62, 55]
[456, 456, 96, 159]
[475, 389, 518, 46]
2136
[18, 20, 34, 55, 10]


In [5]:
file = open('./user_action.p','wb')
pkl.dump(user_action,file)
file.close()

#### 构建rate-matrix
len(user)行，len(concept)列

默认不包含重复项，如果要包含重复项，请修改下面的代码

In [5]:
row = []
col = []
dat = []
for u in user_action:
    for v in user_action[u]:
        row.append(u)
        col.append(v)
        dat.append(1)
rate_matrix = sp.coo_matrix((dat,(row,col)),shape=(len(user_list),len(concept_list)))
rate_matrix = rate_matrix.tocsr()

In [6]:
file = open('./rate_matrix.p','wb')
pkl.dump(rate_matrix,file)
file.close()

#### 构建Adjacency matrix
这里是去除最后一个user action的

In [12]:
row = []
col = []
dat = []
end = []
for u in user_action:
    end.append(user_action[u][-1])
    for v in range(len(user_action[u])-1):
        row.append(u)
        col.append(user_action[u][v])
        dat.append(1)
adjacency_matrix = sp.coo_matrix((dat,(row,col)),shape=(len(user_list),len(concept_list)))
adjacency_matrix = adjacency_matrix.tocsr()

In [14]:
file = open('./adjacency_matrix.p','wb')
pkl.dump(adjacency_matrix,file)
file.close()
file = open('./end_test.p','wb')
pkl.dump(np.array(end),file)
file.close()

#### 构建user的feature
这里先使用one-hot格式，之后换成更复杂的

In [9]:
user_feature = sp.eye(len(user_list))

In [10]:
file = open('./user_feature.p','wb')
pkl.dump(user_feature,file)
file.close()

#### 构建concept-embedding
现在使用word2vec生成的concept-embedding

In [12]:
all_concept_embedding = []
# with open('./all_concept_embedding.txt','r',encoding='utf8') as all_concept_embedding_source:
#     line = all_concept_embedding_source.readline()
#     while line:
#         all_concept_embedding.append(line.strip())
#         line = all_concept_embedding_source.readline()
# print(len(all_concept_embedding))
all_concept_embedding = np.loadtxt('./all_concept_embedding.txt')
all_concept_list = []
with open('./all_concepts_list.txt','r',encoding='utf8') as all_concept_list_source:
    line =  all_concept_list_source.readline()
    while line:
        all_concept_list.append(line.strip())
        line =  all_concept_list_source.readline()
print(len(all_concept_list))

2527


In [13]:
concept_embedding = []
for c in concept_list:
    concept_embedding.append(all_concept_embedding[all_concept_list.index(c)])
print(len(concept_embedding))
# print(concept_embedding[0])

937


In [14]:
file = open('./concept_embedding.p','wb')
pkl.dump(concept_embedding,file)
file.close()

#### 构建Adjacency matrix
user-course的

##### 首先去除敏感信息，变成数字代号

In [13]:
with open('./course_list.p','rb') as source:
    course_list = pkl.load(source)
with open('./user_course.p','rb') as source:
    user_course = pkl.load(source)

user_course_num = {}
for uc in user_course:
    ucn = []
    for c in user_course[uc]:
        ucn.append(course_list.index(c))
    user_course_num[uc] = ucn

In [22]:
row = []
col = []
dat = []
for u in user_course_num:
    for c in user_course_num[u]:
        row.append(u)
        col.append(c)
        dat.append(1)
UC = sp.coo_matrix((dat,(row,col)),shape=(len(user_course_num),len(course_list)))
UC = UC.tocsr()

In [26]:
with open('./UC.p','wb') as source:
    pkl.dump(UC,source)